In [1]:
# importing the necessary packages
import requests, re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time, pdb

coverpage_news = []
website = 'https://www.iasoybeans.com'


##Extract Links
for i in range(5):
    url = website+"/search/news/?q=soybean&p=" + str(i)
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    for li in soup1.find_all('li'):
        h3 = li.findChildren('h3')
        for a in h3:
            a_link = a.find('a')['href']
            #print(a_link)
            coverpage_news.append(a_link)

print("Number of News articles", len(coverpage_news))

Number of News articles 50


In [2]:
##Get data from the website
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []
news_keywords = []
news_published_time = []
source_orgs = []

# Getting the link of the article
for n, link in enumerate(coverpage_news):
    # Reading the content (it is divided in paragraphs)
    link= website+link
    print(link)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    # Getting the title
    soup_article = soup_article.find("article", {"class": "featured-article"})
    title = soup_article.find('h1').get_text() #.find('a')
    print(title)
    list_titles.append(title)
    list_links.append(link)
    # Reading the content (it is divided in paragraphs)
    article_keys = soup_article.find("span").get_text()
    article_published_time = soup_article.find("span").get_text()
    article_published_time = re.sub('\s+',' ',article_published_time).split('|')
    news_keywords.append(article_published_time[1])
    news_published_time.append(article_published_time[0])
    
    body = soup_article.find_all('p')
    final_article = ""

    if len(body) == 0:
        break
    else:
        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(body)):
            #print(p)
            paragraph = body[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        #print(final_article)
        news_contents.append(final_article)

https://www.iasoybeans.com/news/articles/u-s-soybean-exports-to-china-how-low-will-they-go/
U.S. soybean exports to China: How low will they go?
https://www.iasoybeans.com/news/articles/soybean-prices-an-uphill-climb/
Soybean prices: An uphill climb
https://www.iasoybeans.com/news/articles/soybean-leaders-talk-trade-research-sustainability-and-more/
Soybean leaders talk trade, research, sustainability and more
https://www.iasoybeans.com/news/articles/soybean-yields-are-big-in-iowa/
Soybean yields are BIG in Iowa
https://www.iasoybeans.com/news/articles/soybean-yields-influenced-by-dicamba-pleasantly-surprise-farmers/
Soybean yields influenced by dicamba pleasantly surprise farmers
https://www.iasoybeans.com/news/articles/iowa-soybean-association-comments-on-use-of-dicamba/
Iowa Soybean Association comments on use of dicamba
https://www.iasoybeans.com/news/articles/iowa-soybean-association-leaders-recognized-for-conservation-efforts/
Iowa Soybean Association leaders recognized for conse

In [4]:
# Save Data to the dataframe
# ['Article Content', 'Article Title', 'Article Link', 'news_keywords', 'news_published_time']
df_features = pd.DataFrame(
     { 
    'Article Title': list_titles,
    'news_keywords': news_keywords,
    'news_published_time': news_published_time,
    'Article Content': news_contents,
    'Article Link': list_links,
    })

##Save it to the csv file
df_features.to_csv("./newsdata.csv", index=False)
print(df_features.head())

                                       Article Title  \
0  U.S. soybean exports to China: How low will th...   
1                    Soybean prices: An uphill climb   
2  Soybean leaders talk trade, research, sustaina...   
3                     Soybean yields are BIG in Iowa   
4  Soybean yields influenced by dicamba pleasantl...   

                                       news_keywords news_published_time  \
0   Soybean Exports, Policy, Soybean News, Econom...         09/18/2018    
1   Soybean Exports, Policy, Transportation, Soyb...         09/26/2018    
2   Crop Production Research, Soil Health, Water ...         09/24/2018    
3   Soybean Exports, Soybean News, Economics, Wea...         09/19/2018    
4                 Soil Health, Weed Issues, Weather          10/26/2017    

                                     Article Content  \
0  By Matthew Wilde, ISA senior writer   U.S. soy...   
1  By Matthew Wilde, ISA senior writer   Soybean ...   
2  By Lauren Houska, ISA communication